In [3]:
# Module
import numpy as np
import csv
import time

In [43]:
# 난수 생성
np.random.seed(1234)
def randomize():
    np.random.seed(int(time.time()))
#     print(time.time())

# Hyper-parameter
# : 가중치 파라미터 초기화
rnd_mean = 0
rnd_std =0.03
learning_rate = 0.001

1624177220.9945195


In [39]:
# Main Function
# : abalone_exec 함수를 호출하여 1) 데이터셋 로드 > 2) 파라미터 초기화 > 3) 학습 및 평가 수행
# 즉, 딥러닝 모델 생성부터 학습 과정 전체를 abalone_exec 함수로 일괄 처리

def abalone_exec(epoch_cnt=10 , mb_size = 10, report =1):
    # 1) 데이터셋 로드
    load_abalone_dataset()
    # 2) 파라미터 초기화
    init_model()
    # 3) Training & Test
    train_n_test(epoch_cnt,mb_size,report)

In [27]:
# 1) DATA LOAD
def load_abalone_dataset():
    with open ("abalone.csv") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader,None) # 첫행 건너뛰기. 헤더 무시
        rows=[]
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10,1  # 전복 개체들의 입출력 벡터 크기를 10과 1로 지정
    data = np.zeros([len(abalone),input_cnt+ output_cnt])

    for n,row in enumerate(abalone):
        if row[0]=='I' : data [n,0] =1   # 비선형인 성별 정보를 one-hot 벡터 표현으로 변환
        if row[0]=='M' : data [n,1] =1
        if row[0]=='F' : data [n,2] =1            
        data [n,3:] =row[1:]  # 성별 이외의 정보를 일괄 복제

In [46]:
# init_model 함수 정의 : 파라미터 초기화
def init_model():
    global weight, bias, input_cnt,output_cnt
    weight =np.random.normal(rnd_mean,rnd_std,[input_cnt,output_cnt])
    bias=np.zeros([output_cnt])
    print(weight,bias)

init_model()

In [73]:
# train & test set
def train_test(epoch_count,mb_size,report):
    step_count=arrange_data(mb_size)  # 데이터 셔플, train & test 셋 분리, 정렬 작업 수행
    test_x,test_y=get_test_data()     
    
    for epoch in range(epoch_count):  # 인수로 지정된 에포크 수만큼 학습 반복
        losses,accs = [],[]
        
        for n in range(step_count):  # step_count 값만큼 미니배치 처리 반복
            train_x,train_y =get_test_data(mb_size,n)  
            losses,accs = run_train(train_x,train_y)   # train 데이터 가져와 학습
            losses.append(loss)     # 미니배치에서 손실과 정확도 값을 각각 적재
            accs.append(acc)
            
        if report > 0 and (epoch+1)  % report ==0:  # report : 보고주기. 보고주기에 맞으면 손실 및 정확도를 출력
            acc=run_test(train_x,train_y )
            print('Epoch {} : loss = {:5.3f}, /t accuracy ={:5.3f}/{:5.3f}'.format(epoch+1,np.mean(losses),np.mean(accs),acc))
    
    final_acc =run_test(train_x,train_y )  # 최종 테스트 함수 호출 및 값 출력
    print('\nfilnal Test : final accuracy = {:5.3f}'.format(final_acc))
    

In [77]:
# train_test 에서 사용한 함수 정의

def arrange_data(mb_size):    # 데이터 수만큼 일련번호 발생 후 shuffle
    global data , shuffle_map, test_begin_idx  
    shuffle_map=np.arrange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count=int(data.shape[0]*0.8)//mb_size
    test_begin_idx=step_count*mb_size  # train 과 test를 구분하는 경계 인덱스
    
    return step_count


def get_test_data():   
    global data , shuffle_map, test_begin_idx,output_cnt
    test_data=data[shuffle_map[test_begin_idx:]]
    
    return test_data[:,:-output_cnt],test_data[:,-output_cnt:]
    
    

def get_train_data(mb_size,nth):
    global data , shuffle_map, test_begin_idx
    if nth==0:  # nth가 0이면 train set을 부분적으로 섞어 에포크마다 다른 순서로 학습
        np.random.shuffle(shuffle_map[:test_begin_idx])
    trian_data=data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]

    return trian_data[:,:-output_cnt],trian_data[:,-output_cnt:]


# 학습실행 함수와 평가 실행 함수
def run_train(x,y):
    output,aux_nn = forward_neuralnet(x)
    loss, aux_pp =forward_postproc(x)
    accuracy = eval_accuracy(output,y)
    
    G_loss =1.0
    G_output = backprop_postproc(G_loss,aux_pp)
    backprop_neuralnet(G_loss,aux_nn)
    
    return loss,accuracy


def run_test(x,y):
    output,_ =forward_neuralnet(x)
    accuracy=eval_accuracy(output,y)
    
    return accuracy